## LINKTELLER: About the paper

In this paper, we focus on the edge privacy, and consider a training scenario here the data holder Bob with node features will first send training node features to Alice who owns the adjacency information. Alice will then train
a graph neural network (GNN) with the joint information and provide an inference API to Bob. During inference time, Bob is able to provide test node features and query the API to obtain the predictions for test nodes. Under this setting, we first propose a privacy attack LINKTELLER via influence analysis to infer the private edge information held by Alice via designing adversarial queries for Bob.

## Libraries

In [ ]:
!pip -q install --index-url https://download.pytorch.org/whl/cu121 torch==2.3.1

# 2) PyG and companions compiled for torch 2.3.1 + cu121
!pip -q install -f https://data.pyg.org/whl/torch-2.3.1+cu121.html \
  torch_geometric==2.5.3 torch_scatter==2.1.2 torch_sparse==0.6.18

# 3) Pin fsspec to avoid the LocalFileSystem.mv() signature mismatch
!pip -q install --force-reinstall --no-deps fsspec==2023.6.0

# (Optional utilities)
!pip -q install scikit-learn networkx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 99.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 132.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.5 MB/s eta 0:00:00
ERROR: pip's dependency resol

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import to_networkx, dense_to_sparse
from torch_geometric.nn import GCNConv
import networkx as nx
from sklearn.cluster import KMeans
import torch, fsspec, torch_geometric
from torch_geometric.datasets import TUDataset
import numpy as np
from sklearn.model_selection import train_test_split
import math, random
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


device(type='cuda')

## Loading Dataset

In [ ]:
dataset = TUDataset(root='data/TUD', name='MUTAG')  # 188 graphs
sizes = [data.num_nodes for data in dataset]
idx = int(np.argmax([n if n >= 25 else 0 for n in sizes]))  # pick a larger graph
data = dataset[idx]
print(f"Graph index {idx}: nodes={data.num_nodes}, edges={data.num_edges // 2} (undirected)")


Graph index 5: nodes=28, edges=31 (undirected)


Processing...
Done!


## BUild X features

In [ ]:
X = data.x.float()  # [N, D]
N, D = X.shape


K = min(3, len(torch.unique(X, dim=0))) if len(torch.unique(X, dim=0))>1 else 2
km = KMeans(n_clusters=K, n_init=10, random_state=0).fit(X.numpy())
y_node = torch.from_numpy(km.labels_).long()

# Train/val/test node splits
idx_all = np.arange(N)
idx_train, idx_tmp = train_test_split(idx_all, test_size=0.4, random_state=42, stratify=y_node.numpy())
idx_val, idx_test = train_test_split(idx_tmp, test_size=0.5, random_state=42, stratify=y_node.numpy()[idx_tmp])

train_mask = torch.zeros(N, dtype=torch.bool); train_mask[idx_train] = True
val_mask   = torch.zeros(N, dtype=torch.bool); val_mask[idx_val] = True
test_mask  = torch.zeros(N, dtype=torch.bool); test_mask[idx_test] = True

print(f"Splits: train {train_mask.sum().item()}, val {val_mask.sum().item()}, test {test_mask.sum().item()}")


Splits: train 16, val 6, test 6


## Adjacency (A) helpers

In [ ]:
# Edge index is undirected in PyG; keep it as-is
edge_index = data.edge_index  # [2, E]

# For evaluation convenience, build a boolean adjacency (without self loops)
A = torch.zeros((N, N), dtype=torch.bool)
A[edge_index[0], edge_index[1]] = True
A[edge_index[1], edge_index[0]] = True
A.fill_diagonal_(False)
true_edges_undirected = torch.nonzero(torch.triu(A, diagonal=1), as_tuple=False)  # [M, 2]
M_true = true_edges_undirected.shape[0]
density = M_true / (N*(N-1)/2)
print(f"True undirected edges: {M_true} | density={density:.4f}")


True undirected edges: 31 | density=0.0820


## 2 small layer GCN for node classification

In [ ]:
class GCN(nn.Module):
    def __init__(self, in_channels, hidden, out_channels, dropout=0.2):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden)
        self.conv2 = GCNConv(hidden, out_channels)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return x  # logits (N, K)

model = GCN(D, hidden=32, out_channels=K, dropout=0.2).to(device)
X_dev = X.to(device)
edge_index_dev = edge_index.to(device)
y_dev = y_node.to(device)
train_mask_dev = train_mask.to(device)
val_mask_dev   = val_mask.to(device)
test_mask_dev  = test_mask.to(device)


## train the model

In [ ]:
opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
best_val, best_state = -1, None

for epoch in range(400):
    model.train()
    opt.zero_grad()
    logits = model(X_dev, edge_index_dev)
    loss = F.cross_entropy(logits[train_mask_dev], y_dev[train_mask_dev])
    loss.backward()
    opt.step()

    # quick val acc
    model.eval()
    with torch.no_grad():
        val_pred = logits[val_mask_dev].argmax(dim=1)
        val_acc = (val_pred == y_dev[val_mask_dev]).float().mean().item()
    if val_acc > best_val:
        best_val = val_acc
        best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}

print(f"Best val acc: {best_val:.3f}")
model.load_state_dict({k: v for k, v in best_state.items()})


Best val acc: 1.000


<All keys matched successfully>

## Black-box “API” wrapper (returns logits for chosen nodes)

In [ ]:
@torch.no_grad()
def gbb_api(node_ids, X_query):
    """
    node_ids: 1D LongTensor of node indices to fetch from output
    X_query: (N, D) full feature matrix Bob provides (Alice uses it with her private edge_index)
    returns: logits[node_ids] shape (len(node_ids), K)
    """
    model.eval()
    out = model(X_query.to(device), edge_index_dev)  # full-graph forward
    return out[node_ids.to(device)].detach().cpu()


## LINKTELLER influence matrix & scoring

In [ ]:
def influence_matrix_for_v(v, V_I, X_base, delta=1e-2):
    """
    v: node index (int)
    V_I: 1D LongTensor of nodes-of-interest to score against
    X_base: (N, D) baseline features
    returns: Iv (|V_I|, K) = (P' - P)/delta where rows correspond to u in V_I
    """
    node_ids = V_I
    P = gbb_api(node_ids, X_base)

    Xp = X_base.clone()
    Xp[v] = (1.0 + delta) * Xp[v]  # upweight features of v
    Pp = gbb_api(node_ids, Xp)

    Iv = (Pp - P) / delta  # finite-diff approximation
    return Iv  # (|V_I|, K)

def linkteller_scores(V_C, X_base, delta=1e-2):
    """
    V_C: nodes-of-interest (attack surface) as 1D LongTensor
    returns: dict {(u,v): score} for u != v, unordered pairs
    """
    V_C = V_C.cpu()
    scores = {}
    for j, v in enumerate(V_C.tolist()):
        Iv = influence_matrix_for_v(v, V_C, X_base, delta=delta).numpy()  # rows aligned with V_C
        # influence value of v on each u = ||Iv[u,:]||_2
        norms = np.linalg.norm(Iv, axis=1)
        for i, u in enumerate(V_C.tolist()):
            if u == v:
                continue
            key = (min(u,v), max(u,v))
            # symmetrical score: max of v→u and u→v will be handled later; accumulate max
            scores[key] = max(scores.get(key, 0.0), float(norms[i]))
    return scores

# Choose attack node set V_C (we’ll use all nodes to make life easy)
V_C = torch.arange(N, dtype=torch.long)
scores = linkteller_scores(V_C, X, delta=1e-2)

# Turn scores into a sorted list
sorted_pairs = sorted(scores.items(), key=lambda kv: kv[1], reverse=True)
len(sorted_pairs), sorted_pairs[:5]


(378,
 [((16, 17), 1.7930238246917725),
  ((16, 18), 1.7930238246917725),
  ((25, 26), 1.7930220365524292),
  ((25, 27), 1.7930220365524292),
  ((19, 20), 1.793013572692871)])

## Pick top-m pairs using a density belief k̂

In [ ]:
n = N
m_true = M_true
m_belief = int(round(density * (n*(n-1)/2)))

pred_edges = set([pair for (pair, _) in sorted_pairs[:m_belief]])

# ground truth undirected edges as set of tuples (i,j) with i<j
true_edges = set([tuple(e.tolist()) for e in true_edges_undirected])

tp = len(pred_edges & true_edges)
fp = len(pred_edges - true_edges)
fn = len(true_edges - pred_edges)

precision = tp / (tp + fp + 1e-12)
recall    = tp / (tp + fn + 1e-12)
f1        = 2*precision*recall / (precision + recall + 1e-12)
print(f"Precision={precision:.3f} | Recall={recall:.3f} | F1={f1:.3f} | m_belief={m_belief} | true M={m_true}")


Precision=0.710 | Recall=0.710 | F1=0.710 | m_belief=31 | true M=31


##Sweep density belief k̂ to see sensitivity

In [ ]:
def evaluate_at_fraction(frac):
    m = int(round(frac * (n*(n-1)/2)))
    pred = set([pair for (pair, _) in sorted_pairs[:m]])
    tp = len(pred & true_edges)
    fp = len(pred - true_edges)
    fn = len(true_edges - pred)
    p = tp / (tp + fp + 1e-12)
    r = tp / (tp + fn + 1e-12)
    f1 = 2*p*r / (p + r + 1e-12)
    return p, r, f1, m

for frac in [0.5*density, 0.8*density, density, 1.2*density, 1.5*density]:
    p, r, f1, m = evaluate_at_fraction(frac)
    print(f"k_hat={frac:.4f}  m={m:3d}  P={p:.3f} R={r:.3f} F1={f1:.3f}")


k_hat=0.0410  m= 16  P=0.750 R=0.387 F1=0.511
k_hat=0.0656  m= 25  P=0.840 R=0.677 F1=0.750
k_hat=0.0820  m= 31  P=0.710 R=0.710 F1=0.710
k_hat=0.0984  m= 37  P=0.676 R=0.806 F1=0.735
k_hat=0.1230  m= 46  P=0.674 R=1.000 F1=0.805


## Try a different $\Delta$

In [ ]:
scores_different_delta = linkteller_scores(V_C, X, delta=5e-3)
sorted_pairs_2 = sorted(scores_different_delta.items(), key=lambda kv: kv[1], reverse=True)
pred_edges_2 = set([pair for (pair, _) in sorted_pairs_2[:m_belief]])

tp2 = len(pred_edges_2 & true_edges)
fp2 = len(pred_edges_2 - true_edges)
fn2 = len(true_edges - pred_edges_2)
p2 = tp2 / (tp2 + fp2 + 1e-12)
r2 = tp2 / (tp2 + fn2 + 1e-12)
f12 = 2*p2*r2 / (p2 + r2 + 1e-12)
print(f"(Δ=5e-3) Precision={p2:.3f} | Recall={r2:.3f} | F1={f12:.3f}")


(Δ=5e-3) Precision=0.710 | Recall=0.710 | F1=0.710
